In [3]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 15
### Team Member Names: Landon Trinh, Ethan Zemelman, Jessie Deng
### Team Strategy Chosen: SAFE

In [4]:
# Read in CSV ticker file
tickers = pd.read_csv("tickers_example.csv", header=None)
tickers = tickers.rename(columns={0: "ticker"})
tickers.head()

,ticker
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AGN


In [5]:
# Set parameters
start_date = "2023-01-01"
end_date = "2023-10-31"
interval = "1mo"

In [16]:
# Keep stocks that have an average monthly value of 150k shares based on Jan 1, 2023 to Oct 31, 2023
# - A month is defined as a calendar month.  
# - Drop any month that does not have at least 18 trading days. 

def filter_volume(ticker_df):

    
    # If trading days of month < 18 -> drop month
    
    # Get average monthly volume of stock -> make sure its > 150k
    pass

# Get the 1-month average volume for AAPL
# yf.Ticker("AAPL").history(start=start_date, end=end_date, interval=interval).Volume.groupby(pd.Grouper

Date
2023-01-31 00:00:00-05:00    1.443652e+09
2023-02-28 00:00:00-05:00    1.307199e+09
2023-03-31 00:00:00-04:00    1.520267e+09
2023-04-30 00:00:00-04:00    9.697097e+08
2023-05-31 00:00:00-04:00    1.275156e+09
2023-06-30 00:00:00-04:00    1.297101e+09
2023-07-31 00:00:00-04:00    9.960664e+08
2023-08-31 00:00:00-04:00    1.322439e+09
2023-09-30 00:00:00-04:00    1.337587e+09
2023-10-31 00:00:00-04:00    1.172720e+09
Freq: M, Name: Volume, dtype: float64

In [6]:
# Retrieve stock data into DataFrame
def get_stock_data(tickers):
    
    ticker_lst = tickers["ticker"].tolist()
    stock_data = pd.DataFrame()
    
    for ticker in ticker_lst:
        try:
            # Get base currency
            stock_ticker = yf.Ticker(ticker)
            base_currency = stock_ticker.fast_info["currency"]
            
            # Retrieve and store CAD/USD stock data in DataFrame
            if base_currency == "CAD" or base_currency == "USD":
                stock_hist = stock_ticker.history(interval=interval, start=start_date, end=end_date)
                
                # TODO: Make sure we're calculating average monthly volume correctly
                monthly_volume = stock_hist.Volume.mean()
                if monthly_volume < 150_000:
                    print(f"{ticker} has an average monthly volume of {monthly_volume} and will be dropped")
                    continue

                price_data = stock_hist.Close
                price_data.index = price_data.index.strftime("%Y-%m-%d")
                stock_data[ticker] = price_data
        except:
            print(f"{ticker} is not denominated in CAD or USD or de-listed")
        
    return stock_data.dropna()

closing_prices = get_stock_data(tickers)

AGN is not denominated in CAD or USD or de-listed
CELG is not denominated in CAD or USD or de-listed
MON is not denominated in CAD or USD or de-listed
RTN is not denominated in CAD or USD or de-listed


# Stock Analysis

In [7]:
# Get standard deviation of each stock
# Get the beta of each stock
# Get the expected returns of each stock
# Create a correlation matrix of all the stocks

# Portfolio Optimization

In [ ]:
# Create random weights for each stock, and create n number of random portfolios
# Choose the portfolio with the lowest expected returns

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.